
Lear SQLite at tutorialspoint
- https://www.tutorialspoint.com/sqlite/index.htm

SQlite Python interface
- https://docs.python.org/2/library/sqlite3.html

Use flights sample database

Use VScode /SQLite extension to explore db

In [1]:
import sqlite3
conn = sqlite3.connect('example.db')

In [2]:
c = conn.cursor()

In [3]:
# Create table
c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

In [4]:
# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

Query table

In [6]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Do this instead
t = ('RHAT',)
c.execute('SELECT * FROM stocks WHERE symbol=?', t)
print (c.fetchone())

# Larger example that inserts many records at a time
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [7]:
conn.commit()
conn.close()

In [8]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

In [9]:
for r in c.execute('SELECT * FROM stocks order by symbol'):
    print(r)
# print (c.fetchone())

('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)
('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


### in-memory db

In [14]:
import sqlite3
import hashlib

def md5sum(t):
    return hashlib.md5(t.encode('utf-8')).hexdigest()

con = sqlite3.connect(":memory:")
con.create_function("md5", 1, md5sum)
cur = con.cursor()
cur.execute("select md5(?)", ("foo",))
print (cur.fetchone()[0])

acbd18db4cc2f85cedef654fccc4a4d8


In [16]:
md5sum("foo")

'acbd18db4cc2f85cedef654fccc4a4d8'

In [17]:
import sqlite3

class MySum:
    def __init__(self):
        self.count = 0

    def step(self, value):
        self.count += value

    def finalize(self):
        return self.count

con = sqlite3.connect(":memory:")
con.create_aggregate("mysum", 1, MySum)
cur = con.cursor()
cur.execute("create table test(i)")
cur.execute("insert into test(i) values (1)")
cur.execute("insert into test(i) values (2)")
cur.execute("select mysum(i) from test")
print (cur.fetchone()[0])

3


In [18]:
cur.execute("insert into test(i) values (20)")

In [20]:
cur.execute("select mysum(i) from test")
print(cur.fetchall())

[(23,)]


In [22]:
import sqlite3

class IterChars:
    def __init__(self):
        self.count = ord('a')

    def __iter__(self):
        return self

    def next(self):
        if self.count > ord('z'):
            raise StopIteration
        self.count += 1
        return (chr(self.count - 1),) # this is a 1-tuple

In [26]:
con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("create table characters(c)")

In [27]:
theIter = IterChars()
cur.executemany("insert into characters(c) values (?)", theIter.next())

In [28]:
cur.execute("select c from characters")
print (cur.fetchall())

[('a',)]


In [29]:
cur.executemany("insert into characters(c) values (?)", theIter.next())

In [30]:
cur.execute("select c from characters")
print (cur.fetchall())

[('a',), ('b',)]


In [32]:
import sqlite3
import string

# use generator
def char_generator():
    for c in string.ascii_lowercase:
        yield (c,)

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("create table characters(c)")

cur.executemany("insert into characters(c) values (?)", char_generator())

cur.execute("select c from characters")
print (cur.fetchall())

[('a',), ('b',), ('c',), ('d',), ('e',), ('f',), ('g',), ('h',), ('i',), ('j',), ('k',), ('l',), ('m',), ('n',), ('o',), ('p',), ('q',), ('r',), ('s',), ('t',), ('u',), ('v',), ('w',), ('x',), ('y',), ('z',)]


In [33]:
import sqlite3

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.executescript("""
    create table person(
        firstname,
        lastname,
        age
    );

    create table book(
        title,
        author,
        published
    );

    insert into book(title, author, published)
    values (
        'Dirk Gently''s Holistic Detective Agency',
        'Douglas Adams',
        1987
    );
    """)

cur.execute("select * from book")
print (cur.fetchall())

[("Dirk Gently's Holistic Detective Agency", 'Douglas Adams', 1987)]
